# Running and Observing Agents

There are two methods which can be used to run the agents given a task:

- **run()**: It runs the agent given a task as a string or a list of messages and return a `TaskResult`. This method is expected to be run with new message and not complete history.

- **run_stream()**: Same as `run()` bu it returns an iterator of messages that subclasses `BaseChatMessage` or `BaseAgentEvent` followed by `TaskResult` as last message.

In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')

In [2]:
async def web_search(query: str) -> str:
    '''Find information on the web'''
    
    return 'The Labrador is a British breed of gun dog'

In [3]:
agent = AssistantAgent(
    name = 'assistant',
    model_client=openai_model_client,
    tools=[web_search],
    system_message='Use tool to run task'
)

In [5]:
from autogen_agentchat.messages import TextMessage
result = await agent.run(task=TextMessage(content='Find info about labrador', source='user'))
print(result.messages[-1].content)

The Labrador is a British breed of gun dog


In [7]:
result.messages

[TextMessage(id='1c0ce59c-cdc7-41e0-9875-f06d3e6b6b3a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 17, 7, 45, 38, 691246, tzinfo=datetime.timezone.utc), content='Find info about labrador', type='TextMessage'),
 ToolCallRequestEvent(id='c6ed181c-cb9e-45b2-969a-6b904e1d626b', source='assistant', models_usage=RequestUsage(prompt_tokens=62, completion_tokens=17), metadata={}, created_at=datetime.datetime(2025, 7, 17, 7, 45, 42, 738585, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_JPTZOsdKAasN0T735P9oxozp', arguments='{"query":"Labrador information"}', name='web_search')], type='ToolCallRequestEvent'),
 ToolCallExecutionEvent(id='71d2ca09-ba81-4c27-9113-620bc47da49a', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 17, 7, 45, 42, 740585, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The Labrador is a British breed of gun dog', name='web_search', call_id='call_JPTZO

### on_messages()

It is core method to run the agent and gets its response. It provides us with `inner_message`:tool call and `chat_message`: response of the assistant.

In [12]:
from autogen_core import CancellationToken

async def assistant_run() -> None:
    response =await agent.on_messages(
        messages=[TextMessage(content='Find info about labrador', source='user')],
        cancellation_token=CancellationToken()
    )
    
    print(response.inner_messages)
    print("\n\n")
    print(response.chat_message)

await assistant_run()

[ToolCallRequestEvent(id='2d175f7f-464b-44f4-893b-ad964f6c070e', source='assistant', models_usage=RequestUsage(prompt_tokens=290, completion_tokens=24), metadata={}, created_at=datetime.datetime(2025, 7, 18, 5, 27, 29, 931388, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_X0CLcRT5QGJfoNHmyHpsPjCr', arguments='{"query":"Labrador Retriever breed information, characteristics, and care"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='e1d35f20-d002-48bd-85ce-20741a2ac031', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 18, 5, 27, 29, 932516, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The Labrador is a British breed of gun dog', name='web_search', call_id='call_X0CLcRT5QGJfoNHmyHpsPjCr', is_error=False)], type='ToolCallExecutionEvent')]



id='9d98c699-a705-4bae-bcfe-4a573b8e7776' source='assistant' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 18, 5

In [14]:
'''
chat_message=
    ToolCallSummaryMessage(
        id='0690fa46-401a-4c84-80f2-dfe0adc4c180', 
        source='assistant', 
        models_usage=None, 
        metadata={}, 
        created_at=datetime.datetime(2025, 7, 18, 4, 58, 10, 774237, tzinfo=datetime.timezone.utc), 
        content='The Labrador is a British breed of gun dog\nThe Labrador is a British breed of gun dog\nThe Labrador is a British breed of gun dog', 
        type='ToolCallSummaryMessage', 
        tool_calls=[
            FunctionCall(
                id='call_YzzmYwuEOkff1nRR4Cy8r2AE', 
                arguments='{"query": "Labrador breed information"}', 
                name='web_search'), 
            FunctionCall(
                id='call_DVPa5ZiubisIT5S6moQ6ngIV', 
                arguments='{"query": "Labrador dog characteristics"}', 
                name='web_search'), 
            FunctionCall(
                id='call_GC6tLL9xiP1rqlTClmVMZcaE', 
                arguments='{"query": "Labrador health and care"}', 
                name='web_search')
            ], 
        results=[
            FunctionExecutionResult(
                content='The Labrador is a British breed of gun dog', 
                name='web_search', 
                call_id='call_YzzmYwuEOkff1nRR4Cy8r2AE', 
                is_error=False
            ), 
            FunctionExecutionResult(
                content='The Labrador is a British breed of gun dog', 
                name='web_search', 
                call_id='call_DVPa5ZiubisIT5S6moQ6ngIV', 
                is_error=False
            ), 
            FunctionExecutionResult(
                content='The Labrador is a British breed of gun dog', 
                name='web_search', 
                call_id='call_GC6tLL9xiP1rqlTClmVMZcaE', 
                is_error=False)
            ]
    ), 
    
inner_messages=[
    ToolCallRequestEvent(
        id='231c7372-e95e-4f03-ac4e-66ad27bbf45b', 
        source='assistant', 
        models_usage=RequestUsage(prompt_tokens=118, completion_tokens=71), 
        metadata={}, 
        created_at=datetime.datetime(2025, 7, 18, 4, 58, 10, 765191, tzinfo=datetime.timezone.utc), 
        content=[
            FunctionCall(
                id='call_YzzmYwuEOkff1nRR4Cy8r2AE', 
                arguments='{"query": "Labrador breed information"}', 
                name='web_search'
            ), 
            FunctionCall(
                id='call_DVPa5ZiubisIT5S6moQ6ngIV', 
                arguments='{"query": "Labrador dog characteristics"}', 
                name='web_search'
            ), 
            FunctionCall(
                id='call_GC6tLL9xiP1rqlTClmVMZcaE', 
                arguments='{"query": "Labrador health and care"}', 
                name='web_search'
            )
        ], 
        type='ToolCallRequestEvent'
    ), 
    ToolCallExecutionEvent(
        id='8140f539-0e29-422b-b703-b9d2a6280456', 
        source='assistant', 
        models_usage=None, 
        metadata={}, 
        created_at=datetime.datetime(2025, 7, 18, 4, 58, 10, 774237, tzinfo=datetime.timezone.utc), 
        content=[
            FunctionExecutionResult(content='The Labrador is a British breed of gun dog', name='web_search', call_id='call_YzzmYwuEOkff1nRR4Cy8r2AE', is_error=False), FunctionExecutionResult(content='The Labrador is a British breed of gun dog', name='web_search', call_id='call_DVPa5ZiubisIT5S6moQ6ngIV', is_error=False), FunctionExecutionResult(content='The Labrador is a British breed of gun dog', name='web_search', call_id='call_GC6tLL9xiP1rqlTClmVMZcaE', is_error=False)
        ], 
        type='ToolCallExecutionEvent'
    )
]
'''
print()

## Streaming Each Messages

We can also stream each message as it is generated by the agent by using `run_stream()`.

We would be using `Console()` which consumes the messsage stream from `run_stream()` or `on_message_stream()` and renders the messages to the console and returns the last processed `TaskResult` or `Response`.

In [15]:
from autogen_agentchat.ui import Console
async def assistant_run_stream()->None:
    message = TextMessage(content='Find info about labrador',source='user')
    await Console(
        agent.run_stream(task=message),
        output_stats=True       #it enables stats printing 
    )
    
await assistant_run_stream()

---------- TextMessage (user) ----------
Find info about labrador
---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='call_Xs5DzeTbeL0NM0Un3QIqz8NJ', arguments='{"query":"Labrador breed characteristics and care"}', name='web_search')]
[Prompt tokens: 342, Completion tokens: 20]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='The Labrador is a British breed of gun dog', name='web_search', call_id='call_Xs5DzeTbeL0NM0Un3QIqz8NJ', is_error=False)]
---------- ToolCallSummaryMessage (assistant) ----------
The Labrador is a British breed of gun dog
---------- Summary ----------
Number of messages: 4
Finish reason: None
Total prompt tokens: 342
Total completion tokens: 20
Duration: 1.17 seconds
